In [8]:
import wandb
import sys
import matplotlib.pyplot as plt
import scprep
import pandas as pd
sys.path.append('../src/')
# from diffusion import DiffusionModel
# from evaluate import get_results
from omegaconf import OmegaConf
# from main import load_data, make_model
import numpy as np
import os
import glob
import matplotlib.animation as animation
from matplotlib.animation import PillowWriter
import torch
from model2 import Autoencoder, Preprocessor, WDiscriminator
from off_manifolder import offmanifolder_maker
import magic
import torch
import pathlib
import copy

import wandb
import sys
import matplotlib.pyplot as plt
import scprep
import pandas as pd
from omegaconf import OmegaConf
import os
import glob
import numpy as np
import torch
from torch import nn
import torch.optim as optim
from geodesic import jacobian, velocity, CondCurve, GeodesicBridgeOverfit, GeodesicBridge, GeodesicFM
from plotly3d.plot import scatter, trajectories
import torch
from torch import nn
import pytorch_lightning as pl
from procrustes import Procrustes
from torch.utils.data import DataLoader, TensorDataset


import hydra
from omegaconf import DictConfig, OmegaConf
import wandb
from pytorch_lightning.loggers import WandbLogger, TensorBoardLogger
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
import ot as pot

adjoint = False
if adjoint:
    from torchdiffeq import odeint_adjoint as odeint
else:
    from torchdiffeq import odeint

from train import train_model, load_data
from train_dm import load_data as load_data_dm

In [2]:
import sys
from omegaconf import OmegaConf
from hydra import initialize, compose
import yaml


In [3]:
with open('config_ae.yaml', 'r') as file:
    override_config = yaml.safe_load(file)

def dict_to_overrides(d, prefix=''):
    overrides = []
    for k, v in d.items():
        if isinstance(v, dict):
            overrides.extend(dict_to_overrides(v, prefix=f"{prefix}{k}."))
        else:
            overrides.append(f"{prefix}{k}={v}")
    return overrides

overrides = dict_to_overrides(override_config)


In [4]:
initialize(config_path=".")
cfg = compose(config_name="config", overrides=overrides)

/tmp/tmp.6Ory2GY19m/ipykernel_572973/2365906557.py:1: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path=".")


In [5]:
trainloader, valloader, X, phate_coords, colors, preprocessor = load_data(cfg)
cfg.dimensions.data = X.shape[1]
if cfg.training.mode == 'discriminator':
    model = Discriminator(cfg, preprocessor)
elif cfg.training.mode == 'noise_predictor':
    model = NoisePredictor(cfg, preprocessor)
elif cfg.training.mode == 'wdiscriminator':
    model = WDiscriminator(cfg, preprocessor)
elif cfg.training.mode == 'sphere':
    model = SphereEncoder(cfg, preprocessor)
elif cfg.training.mode == 'hyperbolic':
    model = HyperbolicLorenzEncoder(cfg, preprocessor)
elif cfg.training.mode == 'fimm':
    model = FIMMEncoder(cfg, preprocessor)
else:
    model = Autoencoder(cfg, preprocessor)
if cfg.logger.use_wandb:
    logger = WandbLogger()
    checkpoint_callback = ModelCheckpoint(
        dirpath=wandb.run.dir,  # Save checkpoints in wandb directory
        save_top_k=1,  # Save the top 1 model
        monitor=cfg.training.monitor,  # Model selection based on validation loss
        mode='min'  # Minimize validation loss
    )
else:
    logger = TensorBoardLogger(save_dir=os.path.join(cfg.path.root, cfg.path.log))
    checkpoint_callback = ModelCheckpoint(
        dirpath=cfg.path.root,  # Save checkpoints in wandb directory
        filename=cfg.path.model,
        save_top_k=1,
        monitor=cfg.training.monitor,  # Model selection based on validation loss
        mode='min'  # Minimize validation loss
    )
if cfg.training.mode in ['discriminator', 'wdiscriminator', 'noise_predictor', 'sphere', 'hyperbolic', 'fimm']:
    train_model(cfg, model, trainloader, valloader, logger, checkpoint_callback)
elif cfg.training.mode == 'separate':
    cfg.training.mode = 'encoder'
    train_model(cfg, model.encoder, trainloader, valloader, logger, checkpoint_callback)
    model.link_encoder()
    cfg.training.mode = 'decoder'
    train_model(cfg, model.decoder, trainloader, valloader, logger, checkpoint_callback)
elif cfg.training.mode in ['end2end', 'negative', 'radius']: # encoder-only, decoder-only, or end-to-end
    train_model(cfg, model, trainloader, valloader, logger, checkpoint_callback)
elif cfg.training.mode == 'encoder':
    train_model(cfg, model.encoder, trainloader, valloader, logger, checkpoint_callback)
elif cfg.training.mode == 'decoder':
    train_model(cfg, model.decoder, trainloader, valloader, logger, checkpoint_callback)
else:
    raise ValueError('Invalid training mode')


/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/geosink/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'preprocessor' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['preprocessor'])`.
  rank_zero_warn(
/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/geosink/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_env ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A5000') that has Tensor Cores. T

Sanity Checking: 0it [00:00, ?it/s]

/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/geosink/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:488: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(
/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/geosink/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/geosink/lib/python3.11/site-packages/pytorch_lightning/utilities/data.py:83: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we 

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [6]:
with open('config_disc.yaml', 'r') as file:
    override_config = yaml.safe_load(file)

def dict_to_overrides(d, prefix=''):
    overrides = []
    for k, v in d.items():
        if isinstance(v, dict):
            overrides.extend(dict_to_overrides(v, prefix=f"{prefix}{k}."))
        else:
            overrides.append(f"{prefix}{k}={v}")
    return overrides

overrides = dict_to_overrides(override_config)
cfg_disc = compose(config_name="config", overrides=overrides)

In [7]:
trainloader, valloader, X, phate_coords, colors, preprocessor = load_data(cfg_disc)
cfg_disc.dimensions.data = X.shape[1]
if cfg_disc.training.mode == 'discriminator':
    wmodel = Discriminator(cfg_disc, preprocessor)
elif cfg_disc.training.mode == 'noise_predictor':
    wmodel = NoisePredictor(cfg_disc, preprocessor)
elif cfg_disc.training.mode == 'wdiscriminator':
    wmodel = WDiscriminator(cfg_disc, preprocessor)
elif cfg_disc.training.mode == 'sphere':
    wmodel = SphereEncoder(cfg_disc, preprocessor)
elif cfg_disc.training.mode == 'hyperbolic':
    wmodel = HyperbolicLorenzEncoder(cfg_disc, preprocessor)
elif cfg_disc.training.mode == 'fimm':
    wmodel = FIMMEncoder(cfg_disc, preprocessor)
else:
    wmodel = Autoencoder(cfg_disc, preprocessor)
if cfg_disc.logger.use_wandb:
    logger = WandbLogger()
    checkpoint_callback = ModelCheckpoint(
        dirpath=wandb.run.dir,  # Save checkpoints in wandb directory
        save_top_k=1,  # Save the top 1 model
        monitor=cfg_disc.training.monitor,  # Model selection based on validation loss
        mode='min'  # Minimize validation loss
    )
else:
    logger = TensorBoardLogger(save_dir=os.path.join(cfg_disc.path.root, cfg_disc.path.log))
    checkpoint_callback = ModelCheckpoint(
        dirpath=cfg_disc.path.root,  # Save checkpoints in wandb directory
        filename=cfg_disc.path.model,
        save_top_k=1,
        monitor=cfg_disc.training.monitor,  # Model selection based on validation loss
        mode='min'  # Minimize validation loss
    )
if cfg_disc.training.mode in ['discriminator', 'wdiscriminator', 'noise_predictor', 'sphere', 'hyperbolic', 'fimm']:
    train_model(cfg_disc, wmodel, trainloader, valloader, logger, checkpoint_callback)
elif cfg_disc.training.mode == 'separate':
    cfg_disc.training.mode = 'encoder'
    train_model(cfg_disc, wmodel.encoder, trainloader, valloader, logger, checkpoint_callback)
    wmodel.link_encoder()
    cfg_disc.training.mode = 'decoder'
    train_model(cfg_disc, wmodel.decoder, trainloader, valloader, logger, checkpoint_callback)
elif cfg_disc.training.mode in ['end2end', 'negative', 'radius']: # encoder-only, decoder-only, or end-to-end
    train_model(cfg_disc, wmodel, trainloader, valloader, logger, checkpoint_callback)
elif cfg_disc.training.mode == 'encoder':
    train_model(cfg_disc, wmodel.encoder, trainloader, valloader, logger, checkpoint_callback)
elif cfg_disc.training.mode == 'decoder':
    train_model(cfg_disc, wmodel.decoder, trainloader, valloader, logger, checkpoint_callback)
else:
    raise ValueError('Invalid training mode')

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A5000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/geosink/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:613: UserWarning: Checkpoint directory ./results_wd/ exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type         | Params
----------------------------------------------
0 | preprocessor | Preprocessor | 0     
1 | mlp          | MLP   

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
